In [16]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config2

import twelvedata
from twelvedata import TDClient



In [17]:
# set up login
Cpassword = nvda_config2.mysql_password
td = TDClient(apikey=nvda_config2.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'

# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()

# Twelve Data API parameters
symbol = "NVDA, SPCE, ABAT, BB, GME, GRRR, MAXN, TOMZ"
interval = "1min"
outputsize = 5000  

In [18]:
end_date = datetime.now()
start_date = end_date - timedelta(days=1) 
time_batch = td.time_series(symbol=symbol, interval=interval, outputsize=outputsize, start_date=start_date.strftime("%Y-%m-%d"),
        end_date=end_date.strftime("%Y-%m-%d"))
df = time_batch.with_macd().with_macd(fast_period=10).with_stoch().as_pandas()

price_batch = td.price(symbol=symbol)
price_json = price_batch.as_json()
price_df = pd.DataFrame(price_json, index=["price"])
print(price_df.head(5))

df.to_csv('time_data.csv', index=False)
price_df.to_csv('price_data.csv', index=False)


            NVDA     SPCE     ABAT       BB       GME     GRRR     MAXN  \
price  112.24000  6.72500  1.00000  2.41500  23.95000  2.84000  0.21190   

          TOMZ  
price  0.73990  
